In [1]:
import time
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
from SwarmEnvironment import *
from SwarmAgent import *
import numpy
# ------------------------------
# 4. Training Loop
# ------------------------------
#numpy.random.seed(20)
manual_selected_device = "cuda"

n_agents = 10
visible_neighbor_amount = 1
space_size=20
angular_displacement=30
linear_displacement=0.5
goal_error_tolerance = 1
collision_error_tolerance = 0.7
env = SwarmEnv(n_agents=n_agents, space_size=space_size, angular_displacement=angular_displacement, linear_displacement=linear_displacement, visible_neighbor_amount=visible_neighbor_amount)
env.set_random_goals()
agent = Agent(state_dim=env.observation_dimension-3, action_dim=env.action_amount, max_coord = space_size, device=manual_selected_device)
print(env.action_amount)
agent.gamma = 0.95 # q learning gamma, learning rate
agent.epsilon = 1.0 # action randomness 1 for fully random
agent.batch_size = 64

epsilon_decay = 0.995 # action randomness decay rate
epsilon_min = 0.05 # minimum epsilon

env.goal_reward = 100.0
env.collision_reward = -8.0
env.distance_reward_factor = 25
env.step_reward = -0.2

training_steps = 1000
episodes_length = 300

total_rewards = np.zeros(training_steps)
epsilons = np.zeros(training_steps)
time_spend = np.zeros(training_steps)


total_time_start = time.time()
for episode in range(training_steps):
    observations = env.reset()
    total_reward = 0
    start_time = time.time()
    step = 0
    env.collision_count = 0
    for step in range(episodes_length):
       # Batched GPU/MPS inference for all agents
       actions = agent.select_multiple_actions(observations)  # replaces the for-loop

        # Environment step (expects actions as a list or array)
       next_observations, rewards, done, _ = env.step(actions,goal_error_tolerance=goal_error_tolerance, collision_error_tolerance=collision_error_tolerance)

       # Store transitions for all agents
       for i in range(env.n_agents):
           agent.store(observations[i], actions[i], rewards[i], next_observations[i])

       # Train DQN
       agent.train_step()
       agent.update_target() #gradual update target to prevent catastrophic forgetting

       # Move to next step
       observations = next_observations
       total_reward += np.mean(rewards)

       # End early if environment finishes
       if done:
           break



    
    
    delta_time = time.time() - start_time
    total_rewards[episode] = total_reward
    epsilons[episode] = delta_time
    time_spend[episode] = delta_time
    agent.epsilon = max(epsilon_min, agent.epsilon * epsilon_decay)
    #print(f"Episode {episode}, average total reward {total_reward:.2f}, eps {agent.epsilon:.2f}")
    
    print(f"Episode {episode + 1}, steps {step + 1:.0f} (done: {env.done_count/n_agents * 100.0:.3f}% collision: {env.collision_count:.0f}), Average total reward {total_reward:.5f}, epsilon {agent.epsilon:.5f} time {delta_time:.5f}s")

total_time = time.time() - total_time_start
print(f"Total training time {total_time:.5f}s")
torch.save(agent.model.state_dict(), "swarm_agent_model.pth")
torch.save(agent.target.state_dict(), "swarm_target_model.pth")

using device :  cuda
85
Episode 1, steps 300 (done: 10.000% collision: 6), Average total reward -94.71490, epsilon 0.99500 time 1.24363s
Episode 2, steps 300 (done: 0.000% collision: 1), Average total reward -31.94119, epsilon 0.99003 time 0.97054s
Episode 3, steps 300 (done: 0.000% collision: 2), Average total reward -55.15251, epsilon 0.98507 time 0.98882s
Episode 4, steps 300 (done: 10.000% collision: 0), Average total reward 12.64058, epsilon 0.98015 time 1.01516s
Episode 5, steps 300 (done: 10.000% collision: 6), Average total reward 14.11438, epsilon 0.97525 time 1.03550s
Episode 6, steps 300 (done: 10.000% collision: 5), Average total reward 1.57309, epsilon 0.97037 time 1.04727s
Episode 7, steps 300 (done: 0.000% collision: 2), Average total reward 5.19027, epsilon 0.96552 time 1.04881s
Episode 8, steps 300 (done: 10.000% collision: 0), Average total reward 42.75802, epsilon 0.96069 time 1.02342s
Episode 9, steps 300 (done: 0.000% collision: 7), Average total reward 53.58302, e

In [10]:
from swarm_agent_visualization import *

visualize_swarm(agent, env, steps=300, save=True, goal_error_tolerance = 1, collision_error_tolerance = 0.7)

✅ Saved swarm_simulation_normal.gif
✅ Saved swarm_simulation_xy.gif
✅ Saved swarm_simulation_xz.gif
✅ Saved swarm_simulation_yz.gif
